# Structuration DataSet (A ne plus relancer)

In [ ]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# On concaténe les 6 fichiers
import pandas as pd
df_0 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_0.csv")
df_1 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_1.csv")
df_2 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_2.csv")
df_3 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_3.csv")
df_4 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_4.csv")
df_5 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_5.csv")
# df = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron.csv", nrows=1500)
frames = [df_0,df_1,df_2,df_3,df_4,df_5]
df = pd.concat(frames)
# la selection ci-dessous a permis de détecter 3 doublons (toutes colonnes identiques)
# df = df.loc[(df['subject'] == "Sagewood Town Homes")]
print(df.shape)
# df = df_emails.loc[(df['subject'] == "Sagewood Town Homes")]
df.drop('Unnamed: 0', axis=1, inplace=True)
# suppression des doublons (sur toutes les colonnes sauf index)
df.drop_duplicates(subset=['date','sender', 'recipient1', 'subject','text'],keep = 'first', inplace=True)
# on renomme les colonnes
df = df.rename(columns={'sender': 'from', 'recipient1': 'to','subject': 'header', 'text': 'body'})
# et export en csv du résultat.
file_name = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique.csv"
df.to_csv(file_name, encoding='utf-8', index=False)
print(df.shape)
# Taille du fichier divisée par 2 environ

(405968, 6)
(227032, 5)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227032 entries, 0 to 11736
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    227032 non-null  object
 1   from    227032 non-null  object
 2   to      227032 non-null  object
 3   header  218946 non-null  object
 4   body    227032 non-null  object
dtypes: object(5)
memory usage: 10.4+ MB


# Data Preprocessing - cleaning

In [6]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import pandas as pd
# Pour limiter les temps de calcul on travaille uniquement sur les 'nrows' premiers mails
df = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique.csv", nrows=1000)
df.head()

Mounted at /content/drive


,date,from,to,header,body
0,2001-05-14 16:39:00-07:00,phillip.allen@enron.com,tim.belden@enron.com,NaN,"['', 'Here is our forecast', '', ' ']"
1,2001-05-04 13:51:00-07:00,phillip.allen@enron.com,john.lavorato@enron.com,Re:,"['', 'Traveling to have a business meeting tak..."
2,2000-10-18 03:00:00-07:00,phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,"['', 'test successful. way to go!!!']"
3,2000-10-23 06:13:00-07:00,phillip.allen@enron.com,randall.gay@enron.com,NaN,"['', 'Randy,', '', ' Can you send me a schedul..."
4,2000-08-31 05:07:00-07:00,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,"['', ""Let's shoot for Tuesday at 11:45. ""]"


In [7]:
# nbre d'expéditeurs distincts
len(df['from'].unique())

149

In [8]:
# nbre de destinataires distincts
len(df['to'].unique())

231

In [9]:
import numpy as np
df = df.replace('NaN', np.NaN)
# df_emails = df[df['header'].notna()]
df_emails = df
df_emails['NER_header'] = ''
df_emails['NER_body'] = ''
df_emails['body_dict'] = ''
df_emails['summary'] = ''
df_emails.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        1000 non-null   object
 1   from        1000 non-null   object
 2   to          1000 non-null   object
 3   header      784 non-null    object
 4   body        1000 non-null   object
 5   NER_header  1000 non-null   object
 6   NER_body    1000 non-null   object
 7   body_dict   1000 non-null   object
 8   summary     1000 non-null   object
dtypes: object(9)
memory usage: 70.4+ KB


In [10]:
df_emails.columns

Index(['date', 'from', 'to', 'header', 'body', 'NER_header', 'NER_body',
       'body_dict', 'summary'],
      dtype='object')

In [11]:
#
# A ce stade le fichier a la structure nécessaire à Melusine pour le nettoyage
#

In [13]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
spacy_stopwords = nlp.Defaults.stop_words
stopwords = spacy_stopwords
print(spacy_stopwords)

{'front', 'less', 'six', 'most', 'ten', 'amongst', 'hereupon', 'former', 'third', 'against', 'various', 'herein', '‘s', 'neither', 'amount', 'therefore', 'might', 'yourself', 'n‘t', 'why', 'fifty', 'they', 'done', 'not', 'whole', 'my', 'he', 'ca', 'thence', 'within', '’d', 'give', 'that', '‘ll', 'twelve', 'whatever', 'fifteen', 'those', "'d", 'very', 'whether', 'therein', 'becomes', 'towards', 'doing', 'almost', "'ll", 'you', 'nothing', 'between', 'beyond', 'everything', 'does', 'also', 'namely', 'toward', 'whither', 'before', 'anyhow', 'keep', "'s", 'thereafter', 'next', '‘d', 'until', 'been', 'eleven', 'mine', 'yours', 'we', 'did', 'whereby', 'thereupon', 'his', '‘m', 'no', 'onto', 'once', "'ve", '’ve', 'thereby', 'twenty', 'anyone', 'make', 'cannot', 'empty', 'below', 'perhaps', 'anyway', 'again', 'else', 'never', 'can', 'mostly', 'top', 'than', 'only', 'three', 'put', 'the', 'our', 'first', 'itself', 'may', 'if', 'of', 'do', 'above', 'formerly', 'sixty', 'somehow', 'to', 'would', '

# Summary (Extractive method)

In [14]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import re

def untokenize (tokens):
# Suppression des caractres spéciaux ou inutiles \"',]
  # r = re.compile(r"[\\\''\",]")
  r = re.compile(r"[\\\''\",]")
  texte=r.sub('', tokens)
  texte = sent_tokenize(texte)
#  print("Après :\n",type(texte)," ",texte)
  return texte

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:
def remove_stop_words(text,stopwords):
  lst=[]
  for token in str(text).split():
    if token.lower() not in stopwords:    #checking whether the word is not 
      lst.append(token)                    #present in the stopword list.      
  # Join items in the list
#  print("Original text  : ",text)
  result = ' '.join(lst)
#  print("Text after removing stopwords  :   ",result)
  return result

In [16]:
# Transforme Text en Dictionnaire - Key = indice de phrase - Value = Mots de la phrase
import re
from nltk.tokenize import sent_tokenize,word_tokenize

def dictionarize(article):
    dico={}
    phrases=sent_tokenize(article)
    phr2=[]
    for i, phr in enumerate(phrases):
        import re
        phr=re.sub(pattern ="[^a-zA-Z]", repl = ' ', string = phr)
        phr2.append(phr)
    
        phr=word_tokenize(phr)
        phr_lower=[w.lower() for w in phr]
        if len(phr_lower)>4:      
            dico[i]=phr_lower    
    return dico



In [17]:
df_emails.shape[0]

1000

In [18]:
# test unitaire de la fonction dictionarize sur une ligne
#
# artikle = dictionarize(df_emails['body'][32])
# for cle, valeur in artikle.items():
#        print("clé", cle, "vaut", valeur)

In [19]:
import pandas as pd
import numpy as np


def TF(token, artikle):
    """
    Calcule le score TF d'un mot dans un artikle
    
    token : Mot dont le score TF doit être calculé.
    
    artikle : Dictionnaire généré à partir d'un texte.
    """
    f = 0
    for key in artikle:
        for work in artikle[key]:
            if work == token:
                f += 1 
    return np.log(f+1)

def IDF(token, corpus):
    """
    Calcule le score IDF d'un mot dans un corpus d'artikles.
    
    token : Mot dont le score IDF doit être calculé.
    
    corpus : Liste d'artikles.
    """
    N = len(corpus)
    d=0
    present = False
    
    for artikle in corpus:
        for key in artikle:
            if token in artikle[key]:
                present = True
        d += int(present)
        present = False
                
    return np.log(N/(d+1) +1)

def TFIDF(token, artikle, corpus):
    """
    Calcule le score TF-IDF d'un mot dans un texte.
    
    token : mot dont le score doit être calculé.
    
    artikle : artikle qui servira à calculer le score du mot dans cet artikle.
    
    corpus : Liste d'artikles formant le corpus.
    """
    return TF(token, artikle)*IDF(token, corpus)

def score_sentence(corpus, artikle, sentence):
    """
    Calcule le score d'une phrase.
    
    corpus : Liste d'artikles.
    
    artikle : Dictionnaire de phrases.
    
    sentence : Phrase sous forme de liste de mots.
    """
    score_sentence = []
    for word in sentence :
        score_word = TFIDF(word, artikle, corpus)
        score_sentence.append(score_word)
    return np.mean(score_sentence)

def best_sentences(scores_artikle, nb_sentences):
    """
    Retourne les indices des phrases les plus importantes en fonction des scores obtenus.
    
    scores_artikle : Liste des scores de chaque phrase dans un texte.
    
    nb_sentences : Nombre de phrases à sélectionner.
    """
    
    return sorted(np.argsort(scores_artikle)[-nb_sentences:])

def summarize(i, n_sentences, df):
    """
    Synthèse extractive d'un article par la méthode TF-IDF.
    
    i : indice de l'article dans le corpus.
    
    n_sentences : nombre de phrases à conserver.
    
    df : DataFrame contenant les artikles dans une colonne 'Artikle'.    
    """
    corpus = df['body_dict']
    artikle = corpus[i]
    texte=df['body'][i]
    
    
    # Calcul du score de chaque phrase de l'artikle
    scores_artikle = [score_sentence(corpus, artikle, sentence) for sentence in artikle.values()]
 
    
    # Extraction des indices des phrases ayant les meilleurs scores
    result = best_sentences(scores_artikle, n_sentences)
#    print('best sentences :',result)
    
    # Séparation de phrases l'article original 
    tokenized_article = sent_tokenize(texte)
    
    # Extraction des phrases les plus importantes
    print ("i :",i)
    summary = [tokenized_article[i] for i in result]
    print(summary)
    return summary

In [20]:
# Nettoyage : Détokenisation et retokenisation en sentences
deb = 0
fin = df_emails.shape[0]

for id in range(deb,fin):
  text = df_emails['body'].iloc[id]
  text = untokenize(text)
  text = dictionarize(str(text))
  df_emails['body_dict'][id] = text
  # résumé dans le champ 'summarize' en 3 lignes au plus.
  df_emails['summary'][id] = summarize(id, 3, df_emails)


i : 0
[]
i : 1
["['', 'Traveling to have a business meeting takes the fun out of the trip.", 'I would suggest holding the business plan meetings here then take a trip without any formal business meetings.', "', '', 'As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not."]
i : 2
[]
i : 3
["['', 'Randy,', '', ' Can you send me a schedule of the salary and level of everyone in the ', 'scheduling group.", 'Plus your thoughts on any changes that need to be made.', "', '(Patti S for example)', '', 'Phillip']"]
i : 4
['[\'\', "Let\'s shoot for Tuesday at 11:45.  "]']
i : 5
["['', 'Greg,', '', ' How about either next Tuesday or Thursday?"]
i : 6
["['', 'Please cc the following distribution list with updates:', '', 'Phillip Allen (pallen@enron.com)', 'Mike Grigsby (mike.grigsby@enron.com)', 'Keith Holst (kholst@enron.com)', 'Monique Sanchez', 'Frank Ermis', 'John Lavorato', '', '', 

In [21]:
df_emails.shape

(1000, 9)

In [22]:
df_emails['body'][32]

'[\'\', \'---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/26/2000 \', \'11:57 AM ---------------------------\', \'\', \'\', \'"BS Stone" <bs_stone@yahoo.com> on 09/26/2000 04:47:40 AM\', \'To: "jeff" <jeff@freeyellow.com>\', \'cc: "Phillip K Allen" <Phillip.K.Allen@enron.com> \', \'Subject: closing\', \'\', \'\', \'\', \'Jeff, \', \'?\', "Is the closing today?? After reviewing the  agreement?I find it isn\'t binding ", "as far as I can determine.? It is  too vague and it doesn\'t sound like ", \'anything an attorney or title company  would?draft for a real estate \', \'closing--but, of course, I could be  wrong.? \', \'?\', \'If this?closing is going to take place without  this agreement then there is \', "no point in me  following up on this?document\'s validity.? ", \'?\', "I will just need to go back to my closing documents  and see what\'s there and ", \'find out where I am with that and deal with this as  best I can.\', \'?\', \'I guess I was expecting something 

In [23]:
df_emails.shape

(1000, 9)

In [25]:
# texte fonction de résumé sur la ligne 'id'
#id = 12
#print(df_emails['body'][id])
#for cle, valeur in df_emails['body_dict'][id].items():
#        print("Ligne ", cle, "vaut", valeur)
#
#
#print('summary :\n\n',summarize(id, 3, df_emails))
#
# df_emails['summary'].iloc[id] = summarize(id, 5, df_emails)
# print(df_emails['summary'][id])

# SPACY - NER

In [26]:
deb = 0
fin = df_emails.shape[0]
for x in range(deb,fin):
# subject
  subj = df_emails.iloc[x]['header']
#  print(x,' - ',subj)
  doc_subj = nlp(str(subj))
  df_emails['NER_header'][x] = doc_subj.ents

# body
  subj = df_emails.iloc[x]['body']
#  print(x,' - ',subj)
  doc_subj = nlp(str(subj))
  df_emails['NER_body'][x] = doc_subj.ents

In [42]:
df_emails[['header','NER_header','NER_body']][30:40]

,header,NER_header,NER_body
30,NaN,(),"((Phillip, K, Allen, /, HOU, /, ECT), (Richard..."
31,Gas Physical/Financial Position,(),"((Phillip, K, Allen, /, HOU, /, ECT), (12:07, ..."
32,closing,(),"((Phillip, K, Allen, /, HOU, /, ECT), (11:57, ..."
33,NaN,(),"((Chris), (PG&E), (EOL))"
34,NaN,(),"((Phillip, K, Allen, /, HOU, /, ECT), (', 02:0..."
35,NaN,(),"((Phillip, K, Allen, /, HOU, /, ECT), (', 02:0..."
36,NaN,(),"((Keith),)"
37,NaN,(),"((daily), (Mike, Grigsby), (37031), (today), (..."
38,NaN,(),"((Denver), (2002), (2003), (25,000), (2004), (..."
39,NaN,(),"((Jim), (weekly), (California), (this, week))"


In [29]:
df_emails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        1000 non-null   object
 1   from        1000 non-null   object
 2   to          1000 non-null   object
 3   header      784 non-null    object
 4   body        1000 non-null   object
 5   NER_header  1000 non-null   object
 6   NER_body    1000 non-null   object
 7   body_dict   1000 non-null   object
 8   summary     1000 non-null   object
dtypes: object(9)
memory usage: 70.4+ KB


In [30]:
id = 44
doc= nlp(df_emails['header'][id])
displacy.render(doc, style = "ent",jupyter = True)
print("----- eMail ",id," -----")
doc= nlp(df_emails['body'][id])
displacy.render(doc, style = "ent",jupyter = True)
print("\n")



/usr/local/lib/python3.7/dist-packages/spacy/displacy/__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


----- eMail  44  -----


In [ ]:
spacy.explain("PERSON")

In [ ]:
# A ce stade le fichier a la structure attendue par melusine + l'identification des NER par Spacy
df_emails[['header','NER_header','body','NER_body']].iloc[id]

In [ ]:
df_emails['body'].iloc[id]

# SPACY - (Abstractive method)

# SPACY - Summary (Extractive method) - Test n°1



In [77]:
!pip install -U spacy
!python -m spacy download en_core_web_sm
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 8.6 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [97]:
stopwords = list( STOP_WORDS)
# pass document into spacy and store in "doc" object
nlp = spacy.load('en_core_web_sm')

In [100]:
text = df_emails['body'][32]
text

'[\'\', \'---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/26/2000 \', \'11:57 AM ---------------------------\', \'\', \'\', \'"BS Stone" <bs_stone@yahoo.com> on 09/26/2000 04:47:40 AM\', \'To: "jeff" <jeff@freeyellow.com>\', \'cc: "Phillip K Allen" <Phillip.K.Allen@enron.com> \', \'Subject: closing\', \'\', \'\', \'\', \'Jeff, \', \'?\', "Is the closing today?? After reviewing the  agreement?I find it isn\'t binding ", "as far as I can determine.? It is  too vague and it doesn\'t sound like ", \'anything an attorney or title company  would?draft for a real estate \', \'closing--but, of course, I could be  wrong.? \', \'?\', \'If this?closing is going to take place without  this agreement then there is \', "no point in me  following up on this?document\'s validity.? ", \'?\', "I will just need to go back to my closing documents  and see what\'s there and ", \'find out where I am with that and deal with this as  best I can.\', \'?\', \'I guess I was expecting something 

In [101]:
text = df_emails['body_dict'][32]
text

{0: ['forwarded',
  'by',
  'phillip',
  'k',
  'allen',
  'hou',
  'ect',
  'on',
  'am',
  'bs',
  'stone',
  'bs',
  'stone',
  'yahoo',
  'com',
  'on',
  'am',
  'to',
  'jeff',
  'jeff',
  'freeyellow',
  'com',
  'cc',
  'phillip',
  'k',
  'allen',
  'phillip',
  'k',
  'allen',
  'enron',
  'com',
  'subject',
  'closing',
  'jeff'],
 2: ['after',
  'reviewing',
  'the',
  'agreement',
  'i',
  'find',
  'it',
  'isnt',
  'binding',
  'as',
  'far',
  'as',
  'i',
  'can',
  'determine'],
 3: ['it',
  'is',
  'too',
  'vague',
  'and',
  'it',
  'doesnt',
  'sound',
  'like',
  'anything',
  'an',
  'attorney',
  'or',
  'title',
  'company',
  'would',
  'draft',
  'for',
  'a',
  'real',
  'estate',
  'closing',
  'but',
  'of',
  'course',
  'i',
  'could',
  'be',
  'wrong'],
 4: ['if',
  'this',
  'closing',
  'is',
  'going',
  'to',
  'take',
  'place',
  'without',
  'this',
  'agreement',
  'then',
  'there',
  'is',
  'no',
  'point',
  'in',
  'me',
  'following',
 

In [99]:
text = df_emails['summary'][32]
text

["', '?",
 '\', "I will just need to go back to my closing documents  and see what\'s there and ", \'find out where I am with that and deal with this as  best I can.',
 "', 'I guess I was expecting something that would be an  exhibit to a recordable ', 'document or something a little more exact, or  rather?sort of a contract.?"]

In [92]:
doc = nlp(text)
tokens = [token.text for token in doc]
print("tokens : ",tokens)

#List item
# add \n to the punchuvation list
punctuation = punctuation + '\n' + '\n\n'
print("punctuation :",punctuation)

# Preparing a dictionary for word frequencies¶
word_frequencies = {}                       # Dictionary Name
for word in doc:
    if word.text.lower() not in stopwords:
        if word.text.lower() not in punctuation:
            if word.text not in word_frequencies.keys():   
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] +=1
print(word_frequencies)
max_frequcncy = max(word_frequencies.values())

# Normalize the word frencies
for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word]/max_frequcncy
     
#sentence tokenization -Calculate sentences scores¶   By creating a Dictionay for sentences and its normalized frequencies
sentence_tokens = [sent for sent in doc.sents]
print(sentence_tokens)
sentence_scores = {}                                            #create Dictionary
for sent in sentence_tokens:
    for word in sent:
        if word.text.lower() in word_frequencies.keys():        
            if sent not in sentence_scores.keys():              # add word normalized frequencies count in each of these sentences,then with maximum value we are going to find important sentence 
                sentence_scores[sent] = word_frequencies[word.text.lower()]
            else:
                sentence_scores[sent] += word_frequencies[word.text.lower()]
print("sentence_scores :",sentence_scores)

# Can tweek percentage
# Get 30% of important sentences with maximum score - using nlargest
from heapq import nlargest
select_length = int(len(sentence_tokens)*0.4)
select_length
summary = nlargest(select_length , sentence_scores , key = sentence_scores.get)
summary   # Got 4 important sentences
final_summary = [word.text for word in summary]
final_summary
#print(len(final_summary))
#combining above 4 lines togeather to from a summary
summary = ' ' .join(final_summary)
summary
summary1 = summary.replace('\n', ' ')
print('\n')
print("Text summarization using Spacy")
print(summary1)
# Summarization using 'gensim'
from gensim.summarization import summarize
print('\n')
print("Text summarization using gensim")
summarize(text)

tokens :  ['[', "''", ',', "'", '----------------------', 'Forwarded', 'by', 'Phillip', 'K', 'Allen', '/', 'HOU', '/', 'ECT', 'on', '09/26/2000', "'", ',', "'", '11:57', 'AM', '---------------------------', "'", ',', "''", ',', "''", ',', "'", '"', 'BS', 'Stone', '"', '<', 'bs_stone@yahoo.com', '>', 'on', '09/26/2000', '04:47:40', 'AM', "'", ',', "'", 'To', ':', '"', 'jeff', '"', '<', 'jeff@freeyellow.com', '>', "'", ',', "'", 'cc', ':', '"', 'Phillip', 'K', 'Allen', '"', '<', 'Phillip.K.Allen@enron.com', '>', "'", ',', "'", 'Subject', ':', 'closing', "'", ',', "''", ',', "''", ',', "''", ',', "'", 'Jeff', ',', "'", ',', "'", '?', "'", ',', '"', 'Is', 'the', 'closing', 'today', '?', '?', 'After', 'reviewing', 'the', ' ', 'agreement?I', 'find', 'it', 'is', "n't", 'binding', '"', ',', '"', 'as', 'far', 'as', 'I', 'can', 'determine', '.', '?', 'It', 'is', ' ', 'too', 'vague', 'and', 'it', 'does', "n't", 'sound', 'like', '"', ',', "'", 'anything', 'an', 'attorney', 'or', 'title', 'company'

'After reviewing the  agreement?I find it isn\'t binding ", "as far as I can determine.?\nIt is  too vague and it doesn\'t sound like ", \'anything an attorney or title company  would?draft for a real estate \', \'closing--but, of course, I could be  wrong.?'

In [93]:
def summarize_spacy(text):
  doc = nlp(text)
  tokens = [token.text for token in doc]
  print("tokens : ",tokens)

  #List item
  # add \n to the punchuvation list
  punctuation = punctuation + '\n' + '\n\n'
  print("punctuation :",punctuation)

  # Preparing a dictionary for word frequencies¶
  word_frequencies = {}                       # Dictionary Name
  for word in doc:
    if word.text.lower() not in stopwords:
      if word.text.lower() not in punctuation:
        if word.text not in word_frequencies.keys():   
          word_frequencies[word.text] = 1
        else:
          word_frequencies[word.text] +=1
  print(word_frequencies)
  max_frequcncy = max(word_frequencies.values())

# Normalize the word frencies
  for word in word_frequencies.keys():
    word_frequencies[word] = word_frequencies[word]/max_frequcncy
     
#sentence tokenization -Calculate sentences scores¶   By creating a Dictionay for sentences and its normalized frequencies
  sentence_tokens = [sent for sent in doc.sents]
  print(sentence_tokens)
  sentence_scores = {}                                            #create Dictionary
  for sent in sentence_tokens:
    for word in sent:
      if word.text.lower() in word_frequencies.keys():        
        if sent not in sentence_scores.keys():              # add word normalized frequencies count in each of these sentences,then with maximum value we are going to find important sentence 
          sentence_scores[sent] = word_frequencies[word.text.lower()]
        else:
          sentence_scores[sent] += word_frequencies[word.text.lower()]
  print("sentence_scores :",sentence_scores)

  # Can tweek percentage
  # Get 30% of important sentences with maximum score - using nlargest
  from heapq import nlargest
  select_length = int(len(sentence_tokens)*0.4)
  select_length
  summary = nlargest(select_length , sentence_scores , key = sentence_scores.get)
  summary   # Got 4 important sentences
  final_summary = [word.text for word in summary]
  print(final_summary) 
  #print(len(final_summary))
  #combining above 4 lines togeather to from a summary
  summary = ' ' .join(final_summary)
  print(summary)
  summary1 = summary.replace('\n', ' ')
  print('\n')
  print("Text summarization using Spacy")
  print(summary1)
  return (summary1)

In [94]:
print("length of original text:",len(text))
summarize = summarize_spacy (text)
print("length of autosummarization text using Spacy",len(summarize),"= ",str(len(summarize)/len(text)))

print(summarize)
print('\n')

length of original text: 1966
tokens :  ['[', "''", ',', "'", '----------------------', 'Forwarded', 'by', 'Phillip', 'K', 'Allen', '/', 'HOU', '/', 'ECT', 'on', '09/26/2000', "'", ',', "'", '11:57', 'AM', '---------------------------', "'", ',', "''", ',', "''", ',', "'", '"', 'BS', 'Stone', '"', '<', 'bs_stone@yahoo.com', '>', 'on', '09/26/2000', '04:47:40', 'AM', "'", ',', "'", 'To', ':', '"', 'jeff', '"', '<', 'jeff@freeyellow.com', '>', "'", ',', "'", 'cc', ':', '"', 'Phillip', 'K', 'Allen', '"', '<', 'Phillip.K.Allen@enron.com', '>', "'", ',', "'", 'Subject', ':', 'closing', "'", ',', "''", ',', "''", ',', "''", ',', "'", 'Jeff', ',', "'", ',', "'", '?', "'", ',', '"', 'Is', 'the', 'closing', 'today', '?', '?', 'After', 'reviewing', 'the', ' ', 'agreement?I', 'find', 'it', 'is', "n't", 'binding', '"', ',', '"', 'as', 'far', 'as', 'I', 'can', 'determine', '.', '?', 'It', 'is', ' ', 'too', 'vague', 'and', 'it', 'does', "n't", 'sound', 'like', '"', ',', "'", 'anything', 'an', 'attor

UnboundLocalError: ignored

In [68]:

print("length of original text:",len(text))
summarize = summarize_gensim (text)
print("length of autosummarization text using Gensim",len(summarize),"= ",str(len(summarize)/len(text)))

print(summarize)
print('\n')

length of original text: 1966


Text summarization using gensim
length of autosummarization text using Gensim 251 =  0.12767039674465921
After reviewing the  agreement?I find it isn't binding ", "as far as I can determine.?
It is  too vague and it doesn't sound like ", 'anything an attorney or title company  would?draft for a real estate ', 'closing--but, of course, I could be  wrong.?




In [ ]:
df_emails[['header','body','body_dict','summary','NER_body']][30:50]

# SPACY - Summary (Extractive method) - Test n°2

In [102]:
!pip install -U spacy
!python -m spacy download en_core_web_sm
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
stopwords=list(STOP_WORDS)
from string import punctuation
punctuation=punctuation+ '\n'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 3.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [115]:
def summarize_spacy_2 (text):
  # https://www.numpyninja.com/post/text-summarization-through-use-of-spacy-library
  nlp = spacy.load('en_core_web_sm')
  doc= nlp(text)
  tokens=[token.text for token in doc]
  print(tokens)

  # calcul frequence de mots
  word_frequencies={}
  for word in doc:
    if word.text.lower() not in stopwords:
      if word.text.lower() not in punctuation:
        if word.text not in word_frequencies.keys():
          word_frequencies[word.text] = 1
        else:
          word_frequencies[word.text] += 1
  print("\n word_frequencies : ",word_frequencies)

  # normalisation des frequence de mots
  max_frequency=max(word_frequencies.values())
  for word in word_frequencies.keys():
    word_frequencies[word]=word_frequencies[word]/max_frequency
  print("\n Normaized word_frequencies : ",word_frequencies)
  # Sentences token
  sentence_tokens= [sent for sent in doc.sents]
  print(sentence_tokens)
  # Calculate the most important sentences by adding the word frequencies in each sentence.
  sentence_scores = {}
  for sent in sentence_tokens:
    for word in sent:
      if word.text.lower() in word_frequencies.keys():
        if sent not in sentence_scores.keys():                            
          sentence_scores[sent]=word_frequencies[word.text.lower()]
        else:
          sentence_scores[sent]+=word_frequencies[word.text.lower()]

  # identifier 30% du texte avec score maximum
  from heapq import nlargest
  select_length=int(len(sentence_tokens)*0.3)
  select_length
  summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
  final_summary=[word.text for word in summary]
  summary=''.join(final_summary)
  return summary

In [111]:
text = df_emails['summary'][32]
text="""The human coronavirus was first diagnosed in 1965 by Tyrrell and Bynoe from the respiratory tract sample of an adult with a common cold cultured on human embryonic trachea.1 Naming the virus is based on its crown-like appearance on its surface.2 Coronaviruses (CoVs) are a large family of viruses belonging to the Nidovirales order, which includes Coronaviridae, Arteriviridae, and Roniviridae families.3 Coronavirus contains an RNA genome and belongs to the Coronaviridae family.4 This virus is further subdivided into four groups, ie, the α, β, γ, and δ coronaviruses.5 α- and β-coronavirus can infect mammals, while γ- and δ- coronavirus tend to infect birds.6 Coronavirus in humans causes a range of disorders, from mild respiratory tract infections, such as the common cold to lethal infections, such as the severe acute respiratory syndrome (SARS), Middle East respiratory syndrome (MERS) and Coronavirus disease 2019 (COVID-19). The coronavirus first appeared in the form of severe acute respiratory syndrome coronavirus (SARS-CoV) in Guangdong province, China, in 20027 followed by Middle East respiratory syndrome coronavirus (MERS-CoV) isolated from the sputum of a 60-year-old man who presented symptoms of acute pneumonia and subsequent renal failure in Saudi Arabia in 2012.8 In December 2019, a β-coronavirus was discovered in Wuhan, China. The World Health Organization (WHO) has named the new disease as Coronavirus disease 2019 (COVID-19), and Coronavirus Study Group (CSG) of the International Committee has named it as SARS-CoV-2.9,10 Based on the results of sequencing and evolutionary analysis of the viral genome, bats appear to be responsible for transmitting the virus to humans"""
print(text)

The human coronavirus was first diagnosed in 1965 by Tyrrell and Bynoe from the respiratory tract sample of an adult with a common cold cultured on human embryonic trachea.1 Naming the virus is based on its crown-like appearance on its surface.2 Coronaviruses (CoVs) are a large family of viruses belonging to the Nidovirales order, which includes Coronaviridae, Arteriviridae, and Roniviridae families.3 Coronavirus contains an RNA genome and belongs to the Coronaviridae family.4 This virus is further subdivided into four groups, ie, the α, β, γ, and δ coronaviruses.5 α- and β-coronavirus can infect mammals, while γ- and δ- coronavirus tend to infect birds.6 Coronavirus in humans causes a range of disorders, from mild respiratory tract infections, such as the common cold to lethal infections, such as the severe acute respiratory syndrome (SARS), Middle East respiratory syndrome (MERS) and Coronavirus disease 2019 (COVID-19). The coronavirus first appeared in the form of severe acute respi

In [116]:
print("length of original text:",len(text))
summarize = summarize_spacy_2 (text)
print("length of autosummarization text using Spacy",len(summarize),"= ",str(len(summarize)/len(text)))

print(summarize)
print('\n')

length of original text: 1702
['The', 'human', 'coronavirus', 'was', 'first', 'diagnosed', 'in', '1965', 'by', 'Tyrrell', 'and', 'Bynoe', 'from', 'the', 'respiratory', 'tract', 'sample', 'of', 'an', 'adult', 'with', 'a', 'common', 'cold', 'cultured', 'on', 'human', 'embryonic', 'trachea.1', 'Naming', 'the', 'virus', 'is', 'based', 'on', 'its', 'crown', '-', 'like', 'appearance', 'on', 'its', 'surface.2', 'Coronaviruses', '(', 'CoVs', ')', 'are', 'a', 'large', 'family', 'of', 'viruses', 'belonging', 'to', 'the', 'Nidovirales', 'order', ',', 'which', 'includes', 'Coronaviridae', ',', 'Arteriviridae', ',', 'and', 'Roniviridae', 'families.3', 'Coronavirus', 'contains', 'an', 'RNA', 'genome', 'and', 'belongs', 'to', 'the', 'Coronaviridae', 'family.4', 'This', 'virus', 'is', 'further', 'subdivided', 'into', 'four', 'groups', ',', 'ie', ',', 'the', 'α', ',', 'β', ',', 'γ', ',', 'and', 'δ', 'coronaviruses.5', 'α-', 'and', 'β', '-', 'coronavirus', 'can', 'infect', 'mammals', ',', 'while', 'γ-',

# Sauvegarde du fichier final en csv

In [ ]:
df_emails.body[32]

In [ ]:
df_emails.info()

In [ ]:
# SAUVEGARDE en csv du résultat.
file_name = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique_tmp.csv"
df.to_csv(file_name, encoding='utf-8', index=False)